In [8]:
import pandas as pd
import numpy as np
from collections import OrderedDict, namedtuple
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys

# LOAD DATA

In [9]:
google_colab = False

In [10]:
path = './'

if google_colab:
    from google.colab import drive

    drive.mount('/content/drive')
    path = '/content/drive/My Drive/ACADÊMICO/MESTRADO/DISSERTAÇÃO/CHAPTERS/5 EXPERIMENTO/dataset/data_representation_4'


dict_ds = {

    'data_ds3_normal_t1_original' : pd.read_csv(path+'/F16_DS3_normal_t1.csv', header=None),
    'data_ds3_normal_t2_original' : pd.read_csv(path+'/F16_DS3_normal_t2.csv', header=None),
    'data_ds3_fault1_original' : pd.read_csv(path+'/F16_DS3_fault1_leakage.csv', header=None),
    'data_ds3_fault2_original' : pd.read_csv(path+'/F16_DS3_fault2_viscousfriction.csv', header=None),
    'data_ds3_fault3_original' : pd.read_csv(path+'/F16_DS3_fault3_compressibility.csv', header=None),
    'data_ds3_fault4_original' : pd.read_csv(path+'/F16_DS3_fault4_fixedposition.csv', header=None),
}

# EXPERIMENT 1

- Detector Phase with Normalization (only Normal data)

## normalization

In [11]:
ss = StandardScaler()

data_ds3_t1_normal = dict_ds['data_ds3_normal_t1_original']
data_ds3_t2_normal = dict_ds['data_ds3_normal_t2_original']
data_ds3_fault1 = dict_ds['data_ds3_fault1_original']
data_ds3_fault2 = dict_ds['data_ds3_fault2_original']
data_ds3_fault3 = dict_ds['data_ds3_fault3_original']
data_ds3_fault4 = dict_ds['data_ds3_fault4_original']

# fit values
ss.partial_fit(data_ds3_t1_normal)
ss.partial_fit(data_ds3_t2_normal)
ss.partial_fit(data_ds3_fault1)
ss.partial_fit(data_ds3_fault2)
ss.partial_fit(data_ds3_fault3)
ss.partial_fit(data_ds3_fault4)

# transform values
data_ds3_t1_normal = ss.transform(data_ds3_t1_normal)
data_ds3_t2_normal = ss.transform(data_ds3_t2_normal)
data_ds3_fault1 = ss.transform(data_ds3_fault1)
data_ds3_fault2 = ss.transform(data_ds3_fault2)
data_ds3_fault3 = ss.transform(data_ds3_fault3)
data_ds3_fault4 = ss.transform(data_ds3_fault4)

# append normal labels
data_ds3_t1_normal = np.append(data_ds3_t1_normal, np.zeros((data_ds3_t1_normal.shape[0],1)), axis = 1)
data_ds3_t2_normal = np.append(data_ds3_t2_normal, np.zeros((data_ds3_t2_normal.shape[0],1)), axis = 1)

# append fault labels
def generate_fault_label(dataset, fault_label):
    labels = np.array([[fault_label]]*dataset.shape[0])

    return labels

data_ds3_fault1 = np.append(data_ds3_fault1, generate_fault_label(data_ds3_fault1, 1), axis = 1)
data_ds3_fault2 = np.append(data_ds3_fault2, generate_fault_label(data_ds3_fault2, 2), axis = 1)
data_ds3_fault3 = np.append(data_ds3_fault3, generate_fault_label(data_ds3_fault3, 3), axis = 1)
data_ds3_fault4 = np.append(data_ds3_fault4, generate_fault_label(data_ds3_fault4, 4), axis = 1)

## Autoencoder (PyTorch)

In [12]:
# FUNCTIONS AND CLASSES
class Autoencoder(nn.Module):
    def __init__(self, encode_l, decode_l):
        super().__init__()
        self.encoder = nn.Sequential(encode_l)
        self.decoder = nn.Sequential(decode_l)

    def forward(self, x):
        return self.decoder(self.encoder(x))

def run_train(net, train_loader, num_epochs, optimizer, loss_func):
    train_loss = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        losses = []
        for n, (real_samples, _) in enumerate(train_loader):

            net.zero_grad()

            ### forward ###
            if google_colab:
                output = net(real_samples.type(torch.FloatTensor).cuda())
                loss = loss_func(output, real_samples.type(torch.FloatTensor).cuda())
            else:
                output = net(real_samples.type(torch.FloatTensor))
                loss = loss_func(output, real_samples.type(torch.FloatTensor))

            running_loss += loss.item()
            losses.append(loss.item)

            ### backward ###
            loss.backward()
            optimizer.step()

        step_loss = running_loss / len(train_loader)
        train_loss.append(step_loss)
        ### log ###
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss))

    return net, output, train_loss, losses

def generate_ground_truth(data_test):
    dimension = 2700
    ground_truth = []

    for j, x in enumerate(data_test):

        if x.shape[0] == dimension: # X_test
            ground_truth.append(0)
        else: # % others: get column label
            if x[-1] != 0:
                ground_truth.append(1)
            else:
                ground_truth.append(0)

    return ground_truth

def generate_losses(data_test, net, loss_function):
    dimension = 2700
    losses = []
    #regenerate_data = np.zeros((1, dimension))
    for j, x in enumerate(data_test):

        if x.shape[0] > dimension:
            x = x[:-1]

        real = x.reshape(1,-1).astype(np.float32)

        regenerate = net(torch.from_numpy(real))
        #regenerate_data[j] = regenerate.cpu().detach().numpy()

        if google_colab:
            loss_ae = loss_function(regenerate, torch.from_numpy(real).cuda()).cpu().detach().numpy()
        else:
            loss_ae = loss_function(regenerate, torch.from_numpy(real)).cpu().detach().numpy()

        losses.append(loss_ae)

    return losses

def generate_y_hat(losses, loss_threshold):
    y_hat = []

    for l in losses:
        if l < loss_threshold:
            y_hat.append(0)
        else:
            y_hat.append(1)

    return y_hat

def tester(data_test, net, loss_function, loss_threshold = 1):

    ground_truth = []
    losses = []

    for n, dataset_name in enumerate(data_test):
        dataset = data_test[dataset_name]

        ground_truth = ground_truth + generate_ground_truth(dataset)

        losses = losses + generate_losses(dataset, net, loss_function)

    y_hat = generate_y_hat(losses, loss_threshold)

    return confusion_matrix(ground_truth, y_hat, normalize='true'), losses, ground_truth, y_hat

def generate_encode_decode_layers(layers, output_layer):
    od_encode = []
    od_decode = []
    # encode
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_encode.append(('l'+str((len(od_encode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_encode.append(('l'+str((len(od_encode)+1)), nn.ReLU()))

    # decode
    layers.reverse()
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_decode.append(('l'+str((len(od_decode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_decode.append(('l'+str((len(od_decode)+1)), nn.ReLU()))
        else:
            od_decode.append(('l'+str((len(od_decode)+1)), output_layer))

    return OrderedDict(od_encode), OrderedDict(od_decode)

def train(layers, last_layer, lr, epochs, batch_size, X_train, optim, loss_fnc):
    encode_l, decode_l = generate_encode_decode_layers(layers, last_layer)
    net = Autoencoder(encode_l, decode_l)

    if google_colab:
        net.cuda()

    if (optim == 'ADAM'):
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)
    elif(optim == 'SGD'):
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    elif(optim == 'RMSprop'):
        optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)

    torch.manual_seed(111)

    # sets
    train_set = [
        (X_train, X_train) for i in range(len(X_train))
    ]

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True
    )

    # train
    net, output, loss, losses = run_train(net, train_loader, epochs, optimizer, loss_fnc)

    return net, output, loss, losses, loss_fnc

def test(X_train, X_test, net, loss_function):
    faults = {'F1': data_ds3_fault1,
              'F2': data_ds3_fault2,
              'F3': data_ds3_fault3,
              'F4': data_ds3_fault4}

    datas = {'X_TRAIN':X_train,
             'X_TEST':X_test,
             'NORMAL 2':data_ds3_t2_normal}

    datas.update(faults)

    for n, dataset_name in enumerate(datas):
        print('Losses ' + dataset_name , file=log)
        losses = generate_losses(datas[dataset_name], net, loss_function)

        print("mu: ", np.mean(losses), file=log)
        print("std: ", np.std(losses), file=log)

        phi_test = np.mean(losses) + np.std(losses)
        print("phi: ", phi_test, file=log)

        if (dataset_name == 'X_TRAIN'):
            phi = phi_test

        print('******************************', file=log)
        print('******************************', file=log)
        print('******************************', file=log)

    # CONFUSION MATRIX FOR NORMAL_2
    cf, losses, ground_truth, y_hat = tester({'NORMAL 2':data_ds3_t2_normal}, net, loss_function, phi)

    print('NORMAL 2', file=log)
    print(cf, file=log)
    print('******************************', file=log)
    print('******************************', file=log)
    print('******************************', file=log)

    for n, fault_name in enumerate(faults):

        datas = {'X_TEST': X_test,
                 'fault_name': faults[fault_name]}

        cf, losses, ground_truth, y_hat = tester(datas, net, loss_function, phi)

        print('X_TEST x ' + fault_name, file=log)
        print(cf, file=log)
        print('******************************', file=log)
        print('******************************', file=log)
        print('******************************', file=log)



In [ ]:
log = open('output.txt', "a", buffering=1)

X_train, X_test, y_train, y_test = train_test_split(data_ds3_t1_normal[:, :-1], data_ds3_t1_normal[:, -1], test_size=0.7, random_state=42)

"""architectures = [[2700,128,16]]
last_layers = [nn.Sigmoid()]
batch_sizes = [64]
optims = ['ADAM']
losses_fnc = [nn.HingeEmbeddingLoss(), nn.MSELoss()]"""

architectures = [[2700,128,21], [2700,128,16], [2700,128,12], [2700,128,9], [2700,128,6], [2700,128,4], [2700,128,3], [2700,128,2],
                 [2700,512,128,21], [2700,512,128,16], [2700,512,128,12], [2700,512,128,9], [2700,512,128,6], [2700,512,128,3], [2700,512,128,2],
                 [2700,1024,128,21], [2700,1024,128,16], [2700,1024,128,12], [2700,1024,128,9], [2700,1024,128,6], [2700,1024,128,3], [2700,512,128,2],
                 [2700,1024,256,21], [2700,1024,256,16], [2700,1024,256,12], [2700,1024,256,9], [2700,1024,256,6], [2700,1024,256,3], [2700,512,256,2],
                 [2700,1024,512,21], [2700,1024,512,16], [2700,1024,512,12], [2700,1024,512,9], [2700,1024,512,6], [2700,1024,512,3], [2700,512,512,2],
                 [2700,1024,512,64,21], [2700,1024,512,64,16], [2700,1024,512,64,12], [2700,1024,512,64,9], [2700,1024,512,64,6], [2700,1024,512,64,3], [2700,1024,512,64,2],
                 [2700,1024,128,64,21], [2700,1024,128,64,16], [2700,1024,128,64,12], [2700,1024,128,64,9], [2700,1024,128,64,6], [2700,1024,128,64,3], [2700,1024,128,64,2],
                 [2700,1024,256,64,21], [2700,1024,256,64,16], [2700,1024,256,64,12], [2700,1024,256,64,9], [2700,1024,256,64,6], [2700,1024,256,64,3], [2700,1024,128,64,2]]

last_layers = [nn.Sigmoid(),nn.Tanh()]
batch_sizes = [64, 32]
optims = ['ADAM', 'SGD', 'RMSprop']
losses_fnc = [nn.MSELoss() ,nn.L1Loss(), nn.SmoothL1Loss(), nn.BCELoss(), nn.HingeEmbeddingLoss(), nn.KLDivLoss(), nn.TripletMarginLoss(), nn.BCEWithLogitsLoss(), nn.NLLLoss(), nn.PoissonNLLLoss(), nn.MarginRankingLoss()]

for architecture in architectures:
    for optim in optims:
        for batch_size in batch_sizes:
            for last_layer in last_layers:
                for loss_fnc in losses_fnc:
                    print('*****************************************', file=log)
                    print('****************************************', file=log)
                    print('************ HYPERPARAMETERS ************', file=log)
                    print('*****************************************', file=log)
                    print('*****************************************', file=log)
                    print(architecture, file=log)
                    print(last_layer, file=log)
                    print(batch_size, file=log)
                    print(optim, file=log)
                    print(loss_fnc, file=log)
                    print('*****************************************', file=log)
                    print('*****************************************', file=log)

                    try:
                        net, output, loss, losses, loss_function = train(layers=architecture.copy(), last_layer=last_layer, lr=1e-3, epochs=1, batch_size=batch_size, X_train=X_train, optim=optim, loss_fnc=loss_fnc)

                        test(X_train, X_test, net, loss_function)
                    except:
                        print("ERROR: " + ' '.join(map(str,architecture)) + '|' + str(last_layer) + '|' + str(batch_size) + '|' + str(optim) + '|' + str(loss_fnc))
                        print("ERROR: " + ' '.join(map(str,architecture)) + '|' + str(last_layer) + '|' + str(batch_size) + '|' + str(optim) + '|' + str(loss_fnc), file=log)

log.close()

epoch [1/1], loss:0.5185
epoch [1/1], loss:0.4016
epoch [1/1], loss:0.1518
epoch [1/1], loss:0.6139
epoch [1/1], loss:1.0000


C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch [1/1], loss:-0.3354


C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by bot

ERROR: 2700 128 21|Sigmoid()|64|ADAM|TripletMarginLoss()
epoch [1/1], loss:0.8667
ERROR: 2700 128 21|Sigmoid()|64|ADAM|NLLLoss()
epoch [1/1], loss:1.4833
ERROR: 2700 128 21|Sigmoid()|64|ADAM|MarginRankingLoss()
epoch [1/1], loss:0.4081
epoch [1/1], loss:0.3369
epoch [1/1], loss:0.1000
ERROR: 2700 128 21|Tanh()|64|ADAM|BCELoss()
epoch [1/1], loss:1.0000


...

ERROR: 2700 128 21|Tanh()|64|ADAM|TripletMarginLoss()
epoch [1/1], loss:0.6248
ERROR: 2700 128 21|Tanh()|64|ADAM|NLLLoss()
epoch [1/1], loss:0.9114
ERROR: 2700 128 21|Tanh()|64|ADAM|MarginRankingLoss()
epoch [1/1], loss:0.4406
epoch [1/1], loss:0.3434
ERROR: 2700 128 21|Sigmoid()|32|ADAM|SmoothL1Loss()
epoch [1/1], loss:0.3803
epoch [1/1], loss:1.0000
ERROR: 2700 128 21|Sigmoid()|32|ADAM|HingeEmbeddingLoss()


C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch [1/1], loss:-0.3871


C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
C:\Users\MARCELO\anaconda3\lib\site-packages\torch\nn\functional.py:2748: UserWarning: reduction: 'mean' divides the total loss by bot

ERROR: 2700 128 21|Sigmoid()|32|ADAM|TripletMarginLoss()
epoch [1/1], loss:0.7762
ERROR: 2700 128 21|Sigmoid()|32|ADAM|NLLLoss()


In [ ]:
log.close()